In [1]:
!pip install mysql-connector-python

     ---------------------------------------- 15.4/15.4 MB 7.0 MB/s eta 0:00:00


In [14]:
import mysql.connector
import pandas as pd

In [15]:
conn = mysql.connector.connect(
    host="localhost",
    database="db",
    user="user",
    password="password"
)

cursor = conn.cursor()

### Select all customers that have multiple active loans

In [16]:
cursor.execute('''SELECT
                    id,
                    surname,
                    birthday,
                    address,
                    phone_number,
                    COUNT(*) AS contratos
                FROM (
                SELECT c.id, c.surname, c.birthday, c.address, c.phone_number
                FROM clientes c
                inner JOIN contratos con ON c.id = con.id
                WHERE con.Contract_end < CURDATE()
                ) as contract_n 
                GROUP BY
                    id,
                    surname,
                    birthday,
                    address,
                    phone_number''')

results = cursor.fetchall()
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(results, columns=columns)
df.head()

,id,surname,birthday,address,phone_number,contratos
0,362,Tailor,1998-04-03,292 Bellgrove Point,200-244-9359,3
1,476,Nero,1998-08-26,63 Linden Avenue,836-469-6127,2
2,318,Edgar,1996-03-22,481 Mosinee Plaza,927-610-5335,3
3,405,Shaina,1995-06-15,958 Glacier Hill Trail,664-193-2402,1
4,454,Barret,1997-02-13,8 Ohio Drive,346-229-8346,2


## Select all customers that have made two consecutive payments of any of their active loans.

In [17]:

cursor.execute('''WITH ranked_payments AS (
                        SELECT 
                        id,
                        payment_dates,
                        status,
                        (
                            SELECT status
                            FROM pagos p2
                            WHERE p2.id = p1.id AND p2.payment_dates > p1.payment_dates
                            ORDER BY p2.payment_dates
                            LIMIT 1
                        ) AS next_status
                    FROM pagos p1
                    order by id, payment_dates asc
                    ),
                    filtered as (
                        select * from ranked_payments rp WHERE
                        rp.status = 'make' AND rp.next_status = 'make'
                    ),
                    specific_clients as (
                    SELECT  
                        c.id,
                        c.surname,
                        c.birthday,
                        c.address,
                        c.phone_number
                    FROM
                        clientes c
                    JOIN
                        ranked_payments rp ON c.id = rp.id
                    WHERE
                        rp.status = 'make' AND rp.next_status = 'make'
                    ORDER BY
                        rp.id, rp.payment_dates
                    )
                    select DISTINCT * from specific_clients;
                    ''')

results = cursor.fetchall()
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(results, columns=columns)
df.head()

,id,surname,birthday,address,phone_number
0,1,Caryl,1996-01-27,2 Farmco Crossing,595-642-5891
1,3,Aprilette,1995-01-16,697 Sachs Avenue,859-366-3566
2,4,Dorree,1998-12-30,33 Mayfield Way,883-884-4505
3,5,Darbee,1998-05-17,969 Loeprich Trail,725-355-0696
4,6,Hallie,1998-02-07,9 Elgar Road,519-458-3990


## Select the total amount paid and the average number of payments made by customers that have only one active loan.

In [18]:

cursor.execute('''with contracts as (
                    SELECT
                        id,
                        -- ids,
                        surname,
                        birthday,
                        address,
                        phone_number,
                        sum(payment_amount) as total_payment,
                        COUNT(*) AS contratos
                    FROM (
                    SELECT c.id, c.surname, c.birthday, c.address, c.phone_number, con.payment_amount
                    -- con.payment_amount, con.id as ids
                    FROM clientes c
                    inner JOIN contratos con ON c.id = con.id
                    WHERE con.Contract_end < CURDATE()
                    ) as contract_n 
                    GROUP BY
                        id,
                        -- ids,
                        surname,
                        birthday,
                        address,
                        phone_number
                    ) select id,
                        surname,
                        birthday,
                        address,
                        phone_number,
                        -- total_payment,
                        count(*) as payments,
                        ROUND(total_payment * COUNT(*),2) AS total_payment_payments
                    from (Select c.*, p2.status from contracts c inner join pagos p2 on p2.id=c.id where c.contratos in (1) and p2.status='make') as numberpay 
                    GROUP BY
                        id,
                        surname,
                        birthday,
                        address,
                        phone_number''')

results = cursor.fetchall()
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(results, columns=columns)
df.head()

,id,surname,birthday,address,phone_number,payments,total_payment_payments
0,405,Shaina,1995-06-15,958 Glacier Hill Trail,664-193-2402,11,8195.85
1,276,Abeu,1996-11-15,3 Eagan Trail,150-200-0668,42,114826.38
2,166,Ahmad,1997-12-24,18 Welch Lane,312-526-2377,19,50494.08
3,322,James,1996-01-08,1643 Reindahl Street,464-268-2626,43,13144.18
4,103,Creight,1998-04-09,13 Milwaukee Plaza,463-139-7117,33,164182.07


## Select the total amount paid grouped by zip code and age buckets

In [19]:
cursor.execute('''SELECT
                    age, 
                    zip,
                    sum(total_amount)
                FROM (
                SELECT c.id, c.surname, c.age, c.zip, con.total_amount
                -- con.payment_amount, con.id as ids
                FROM clientes c
                inner JOIN contratos con ON c.id = con.id
                WHERE con.Contract_end < CURDATE()
                ) as contract_n 
                GROUP BY
                    age,zip
                    ''')

results = cursor.fetchall()
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(results, columns=columns)
df.head()

,age,zip,sum(total_amount)
0,26,51754,297896
1,28,51647,447110
2,29,51765,19372
3,27,51695,411498
4,28,51697,311026


In [20]:
cursor.close()
conn.close()